In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("final_project") \
    .master("local[*]")\
    .config("spark.executor.memory", "2g")\
    .config("spark.jars", "/usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.5.0.jar") \
    .getOrCreate()

23/10/27 10:19:08 WARN Utils: Your hostname, ubuntu-Lenovo-Legion-5-15ARH05 resolves to a loopback address: 127.0.1.1; using 172.16.5.112 instead (on interface wlp4s0)
23/10/27 10:19:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/10/27 10:19:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/27 10:19:08 WARN DependencyUtils: Local jar /usr/lib/jvm/java-11-openjdk-amd64/lib/postgresql-42.5.0.jar does not exist, skipping.
23/10/27 10:19:09 INFO SparkContext: Running Spark version 3.4.1
23/10/27 10:19:09 INFO ResourceUtils: ==============================================================
23/10/27 10:19:09 INFO ResourceUtils: No custom resources configured for spark.driver.
23/10/27 10:19:09 INFO ResourceUtils: ==============================================================
23/10/27 10:19:09 INFO SparkContext: Submitted application: final_project
23/10/27 10:19:09 INFO ResourceProfi

In [5]:
csv = "/home/ubuntu/Desktop/final_project/Raw_Data/job_descriptions.csv"


In [6]:
df = spark.read.csv(csv, header=True, inferSchema=False)

23/10/27 10:19:49 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
23/10/27 10:19:49 INFO SharedState: Warehouse path is 'file:/home/ubuntu/Desktop/final_project/ETL/spark-warehouse'.
23/10/27 10:19:50 INFO InMemoryFileIndex: It took 54 ms to list leaf files for 1 paths.
23/10/27 10:19:50 INFO InMemoryFileIndex: It took 2 ms to list leaf files for 1 paths.
23/10/27 10:19:53 INFO FileSourceStrategy: Pushed Filters: 
23/10/27 10:19:53 INFO FileSourceStrategy: Post-Scan Filters: (length(trim(value#0, None)) > 0)
23/10/27 10:19:53 INFO CodeGenerator: Code generated in 221.305194 ms
23/10/27 10:19:53 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 200.2 KiB, free 434.2 MiB)
23/10/27 10:19:53 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 34.6 KiB, free 434.2 MiB)
23/10/27 10:19:53 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on 172.16.5.112:41715 (size: 3

In [6]:
df.show()

+----------------+-------------+--------------+------------+--------------------+--------------------+--------+---------+---------+------------+----------------+----------+------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|          Job Id|   Experience|Qualifications|Salary Range|            location|             Country|latitude|longitude|Work Type|Company Size|Job Posting Date|Preference|    Contact Person|             Contact|           Job Title|                Role|         Job Portal|     Job Description|            Benefits|              skills|    Responsibilities|             Company|     Company Profile|
+----------------+-------------+--------------+------------+--------------------+--------------------+--------+---------+---------+------------+----------------+----------+------------------+-------

In [7]:
df.persist()

23/10/27 10:19:59 INFO FileSourceStrategy: Pushed Filters: 
23/10/27 10:19:59 INFO FileSourceStrategy: Post-Scan Filters: 


DataFrame[Job Id: string, Experience: string, Qualifications: string, Salary Range: string, location: string, Country: string, latitude: string, longitude: string, Work Type: string, Company Size: string, Job Posting Date: string, Preference: string, Contact Person: string, Contact: string, Job Title: string, Role: string, Job Portal: string, Job Description: string, Benefits: string, skills: string, Responsibilities: string, Company: string, Company Profile: string]

In [8]:
dropped = ["Contact Person", "Contact", "Benefits","Company Profile"]

In [9]:
df = df.drop(*dropped)

In [10]:
df.show()

+----------------+-------------+--------------+------------+--------------------+--------------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|          Job Id|   Experience|Qualifications|Salary Range|            location|             Country|latitude|longitude|Work Type|Company Size|Job Posting Date|Preference|           Job Title|                Role|         Job Portal|     Job Description|              skills|    Responsibilities|             Company|
+----------------+-------------+--------------+------------+--------------------+--------------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|1089843540111562|5 to 15 Years|        M.T

In [10]:
df.printSchema()

root
 |-- Job Id: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Qualifications: string (nullable = true)
 |-- Salary Range: string (nullable = true)
 |-- location: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- Work Type: string (nullable = true)
 |-- Company Size: string (nullable = true)
 |-- Job Posting Date: string (nullable = true)
 |-- Preference: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Role: string (nullable = true)
 |-- Job Portal: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- Responsibilities: string (nullable = true)
 |-- Company: string (nullable = true)



In [11]:
from pyspark.sql.types import LongType, FloatType, IntegerType, DateType

In [12]:
df = df.withColumn("Job Id", df["Job Id"].cast(LongType()))\
        .withColumn("latitude", df["latitude"].cast(FloatType()))\
         .withColumn("longitude", df["longitude"].cast(FloatType()))\
          .withColumn("Company Size", df["Company Size"].cast(IntegerType()))\
           .withColumn("Job Posting Date", df["Job Posting Date"].cast(DateType()))

In [14]:
import pyspark.sql.functions as F
import re

def calculate_average(range_str):
    # Use regular expression to extract numbers
    numbers = re.findall(r'\d+', range_str)
    if len(numbers) == 2:
        lower = int(numbers[0])
        upper = int(numbers[1])
        avg = (lower + upper) / 2
        return avg
    else:
        return None

calculate_average_udf = F.udf(calculate_average)

# Add a new column with the calculated average
new_df = df.withColumn("Average", calculate_average_udf(df["Salary Range"]))

In [16]:
new_df.withColumnRenamed("Average","Average Salary").printSchema()


root
 |-- Job Id: long (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Qualifications: string (nullable = true)
 |-- Salary Range: string (nullable = true)
 |-- location: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- Work Type: string (nullable = true)
 |-- Company Size: integer (nullable = true)
 |-- Job Posting Date: date (nullable = true)
 |-- Preference: string (nullable = true)
 |-- Job Title: string (nullable = true)
 |-- Role: string (nullable = true)
 |-- Job Portal: string (nullable = true)
 |-- Job Description: string (nullable = true)
 |-- skills: string (nullable = true)
 |-- Responsibilities: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Average Salary: string (nullable = true)



In [16]:
# from pyspark.sql.functions import regexp_replace, split, col

# # Assuming you have a DataFrame 'df' with a 'Salary Range' column
# # Remove the '$' sign and split the range into two columns
# new_new_df = df.withColumn("Salary Range", regexp_replace("Salary Range", "[$,K]", ""))
# split_col = split(df["Salary Range"], "-")

# # Extract minimum and maximum salary values and calculate the average
# new_new_df = new_new_df.withColumn("Min Salary", split_col.getItem(0).cast("int"))
# new_new_df = new_new_df.withColumn("Max Salary", split_col.getItem(1).cast("int"))
# new_new_df = new_new_df.withColumn("Average Salary", (col("Min Salary") + col("Max Salary")) / 2)

# # Show the updated DataFrame
# new_new_df.show()

AnalysisException: Resolved attribute(s) Salary Range#43 missing from Job Id#221L,Experience#41,Qualifications#42,Salary Range#1445,location#44,Country#45,latitude#241,longitude#261,Work Type#48,Company Size#281,Job Posting Date#301,Preference#51,Job Title#54,Role#55,Job Portal#56,Job Description#57,skills#59,Responsibilities#60,Company#61 in operator !Project [Job Id#221L, Experience#41, Qualifications#42, Salary Range#1445, location#44, Country#45, latitude#241, longitude#261, Work Type#48, Company Size#281, Job Posting Date#301, Preference#51, Job Title#54, Role#55, Job Portal#56, Job Description#57, skills#59, Responsibilities#60, Company#61, cast(split(Salary Range#43, -, -1)[0] as int) AS Min Salary#1465]. Attribute(s) with the same name appear in the operation: Salary Range. Please check if the right attribute(s) are used.;
!Project [Job Id#221L, Experience#41, Qualifications#42, Salary Range#1445, location#44, Country#45, latitude#241, longitude#261, Work Type#48, Company Size#

In [17]:
##Load the clean data in postgres

new_df.write.format('jdbc').options(url='jdbc:postgresql://localhost:5432/final_project',driver = 'org.postgresql.Driver', dbtable = 'job_description_clean', user="postgres",password="postgres" ).mode('overwrite').save()

23/10/27 10:25:39 INFO SparkContext: Starting job: save at NativeMethodAccessorImpl.java:0
23/10/27 10:25:39 INFO DAGScheduler: Got job 2 (save at NativeMethodAccessorImpl.java:0) with 16 output partitions
23/10/27 10:25:39 INFO DAGScheduler: Final stage: ResultStage 2 (save at NativeMethodAccessorImpl.java:0)
23/10/27 10:25:39 INFO DAGScheduler: Parents of final stage: List()
23/10/27 10:25:39 INFO DAGScheduler: Missing parents: List()
23/10/27 10:25:39 INFO DAGScheduler: Submitting ResultStage 2 (MapPartitionsRDD[31] at save at NativeMethodAccessorImpl.java:0), which has no missing parents
23/10/27 10:25:39 INFO MemoryStore: Block broadcast_5 stored as values in memory (estimated size 55.0 KiB, free 177.2 MiB)
23/10/27 10:25:39 INFO MemoryStore: Block broadcast_5_piece0 stored as bytes in memory (estimated size 23.1 KiB, free 177.2 MiB)
23/10/27 10:25:39 INFO BlockManagerInfo: Added broadcast_5_piece0 in memory on 172.16.5.112:41715 (size: 23.1 KiB, free: 177.5 MiB)
23/10/27 10:25:39

In [15]:
new_df.show(5)

+----------------+-------------+--------------+------------+----------+----------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------+
|          Job Id|   Experience|Qualifications|Salary Range|  location|         Country|latitude|longitude|Work Type|Company Size|Job Posting Date|Preference|           Job Title|                Role|  Job Portal|     Job Description|              skills|    Responsibilities|             Company|Average|
+----------------+-------------+--------------+------------+----------+----------------+--------+---------+---------+------------+----------------+----------+--------------------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+-------+
|1089843540111562|5 to 15 Years|        M.Tech|   $59K-$99K|   Douglas|     Isle o